In [180]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import Adam
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import argparse
import os
import numpy as np
import joblib
import glob
import h5py

In [2]:
def load_data(files):

    data = pd.concat([pd.read_csv(file) for file in files], ignore_index=True)
    
    # Define features and targets
    feature_columns = ["head", "neck", "left_ear", "right_ear", "left_shoulder", "left_elbow", "left_hand", "right_shoulder", "right_elbow", "right_hand"]
    
    
    def parse_feature(value):
        if isinstance(value, str) and value.startswith("(") and value.endswith(")"):
            # Extract the first number from the tuple-like string
            return float(value.split(",")[0].strip("()"))
        return float(value)  # If already a float
    
    X = data[feature_columns].applymap(parse_feature)
    y = data['hand_raised']

    encoder = LabelEncoder()
    ey =    encoder.fit_transform(y).reshape(-1,1)
   
    print("X:", X)

    return X, ey

In [181]:
# For two inputs data loader
def load_data_2in(files):

    data = pd.concat([pd.read_csv(file) for file in files], ignore_index=True)
    
    feature_columns = ["head", "neck", "left_ear", "right_ear", "left_shoulder", "left_elbow", "left_hand", "right_shoulder", "right_elbow", "right_hand"]
    
    def parse_feature(value):
        if isinstance(value, str) and value.startswith("(") and value.endswith(")"):
            # Extract the first number from the tuple-like string
            # x,y = float(value.split(",")[0].strip("()"))
            x,y = value.strip("()").split(",")
            return float(x),float(y)
        else: 
            return float(value), float(0)  # If already a float
    
    X_x = pd.DataFrame()
    X_y = pd.DataFrame()
    for feature in feature_columns:
        X_x[feature], X_y[feature] = zip(*data[feature].map(parse_feature))
    yt = data['hand_raised']

    X_combined = pd.concat([X_x, X_y], axis=1)

    encoder = LabelEncoder()
    ey =    encoder.fit_transform(yt).reshape(-1,1)
   
    # print("X:", X_combined)
    # print("X_y:", X_y)
    # print("y:", yc, yc.shape, type(yc))

    return X_combined, ey

In [196]:
from sklearn import tree

def main(folder_path, train_files, val_files, rs):

    # Resolve full paths for training and validation files
    train_file_paths = [os.path.join(folder_path, file) for file in train_files]

    # Load training and validation data
    # X_train, y_train = load_data(train_file_paths)
    X_train, y_train = load_data_2in(train_file_paths)

    # early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    # rf_model = RandomForestClassifier(n_estimators=nest, random_state=rs)
    # rf_model.fit(X_train,y_train)
    rf_model = RandomForestClassifier()

    param_grid = {
        'n_estimators': [50, 75, 100],  # Number of trees
        'criterion': ['gini', 'entropy','log_loss'],
        'max_depth': [None, 10, 20, 30, 40],
        'min_samples_split': [2, 5],
        'random_state': [42,35]        
    }

    kfold = KFold(n_splits=10, shuffle=True, random_state=rs)

    grid_search = GridSearchCV(
        estimator=rf_model, 
        param_grid=param_grid, 
        cv=kfold, 
        scoring='recall', 
        verbose=2, 
        n_jobs=-1  # Use all available CPU cores
    )
    grid_search.fit(X_train, y_train)

    print("Best Hyperparameters:", grid_search.best_params_)
    best_model = grid_search.best_estimator_

    # dc_model = tree.DecisionTreeClassifier(criterion="gini", random_state=38)
    # dc_model.fit(X_train, y_train)
    # print(dc_model)
    
    # dr_model = tree.DecisionTreeRegressor(criterion="squared_error", random_state=38)
    # dr_model.fit(X_train, y_train)


    val_file_paths = [os.path.join(folder_path, file) for file in val_files]
    # X_val, y_val = load_data(val_file_paths)
    X_val, y_val = load_data_2in(val_file_paths)
    # y_pred = rf_model.predict(X_val)
    y_pred = best_model.predict(X_val)

    accuracy = accuracy_score(y_val,y_pred)
    print("accuracy: ", accuracy)
    print("report", classification_report(y_val,y_pred))

    return best_model

training_files = [
    "v1.csv", "v3.csv", "v4.csv", "v5.csv", "v7.csv",  
    "v8.csv", "v9.csv", "v10.csv", "v11.csv", "v15.csv", 
    "v16.csv", "v18.csv", "v19.csv", "v20.csv", "v21.csv", 
    "v22.csv", "v23.csv", "v24.csv", "v26.csv", "v27.csv"
]

validation_files = ["v2.csv", "v12.csv", "v13.csv", "v14.csv", "v25.csv"]

rf_model = main("C:/OsamaEjaz/Qiyas_Gaze_Estimation/Wajahat_Yolo_keypoint/handraisedataset/normalize", training_files, validation_files, 42 )
rf_model

Fitting 10 folds for each of 180 candidates, totalling 1800 fits


c:\Anaconda\envs\ypose\lib\site-packages\sklearn\model_selection\_search.py:1103: UserWarning: One or more of the test scores are non-finite: [        nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan         nan    

Best Hyperparameters: {'criterion': 'gini', 'max_depth': None, 'min_samples_split': 2, 'n_estimators': 50, 'random_state': 42}
accuracy:  0.7513762642427346
report               precision    recall  f1-score   support

           0       0.75      0.94      0.83      4956
           1       0.80      0.43      0.56      2855
           2       0.00      0.00      0.00         0

    accuracy                           0.75      7811
   macro avg       0.52      0.46      0.46      7811
weighted avg       0.77      0.75      0.73      7811



c:\Anaconda\envs\ypose\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Anaconda\envs\ypose\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Anaconda\envs\ypose\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


RandomForestClassifier(n_estimators=50, random_state=42)

In [186]:
rf_model

RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=50,
                       random_state=42)

In [185]:
# joblib.dump(rf_model, "./rf_1in.joblib")
joblib.dump(rf_model, "./rf_best1.joblib")

['./rf_best1.joblib']

In [53]:
model_rf = joblib.load("./dc_gen.joblib")
model_rf


DecisionTreeClassifier(random_state=38)

In [189]:
from ultralytics import YOLO
import cv2

person_ranges = {
    "Person1": lambda x: 90 < x < 367,
    "Person2": lambda x: 370 <= x <= 730,
    "Person3": lambda x: 740 <= x <= 1080,
    "Person4": lambda x: x > 1080
}

# def keypoint_model(frame,model_file):
def keypoint_model(frame):

    keypoint_labels = ["head", "neck", "left_ear", "right_ear", "left_shoulder",
                    "left_elbow", "left_hand", "right_shoulder", "right_elbow", "right_hand"]

    model = YOLO("runs/pose/trail4/weights/best_y11.pt")
    frame = cv2.resize(frame, (1280, 720))
    results = model(frame)
    keypoints_dict = {person: {kp_idx: {'x': 0, 'y': 0} for kp_idx in range(len(keypoint_labels))} for person in person_ranges.keys()}

    # Iterate over each detected object and process their keypoints
    for result in results:
        keypoints = result.keypoints  # Access the keypoints object

        if keypoints is not None:
            keypoints_data = keypoints.data
            for person_keypoints in keypoints_data:
                for kp_idx, keypoint in enumerate(person_keypoints[:len(keypoint_labels)]):
                    x, y = keypoint[0].item(), keypoint[1].item()
                    for person, condition in person_ranges.items():
                        if condition(x):
                                keypoints_dict[person][kp_idx] = {'x': round(x, 2), 'y': round(y, 2)}
     
    print("keypoints", keypoints_dict)
    return keypoints_dict

In [192]:
frame_width = 1280
frame_height = 780

def parse_and_normalize(keypoint, person,subtraction_values):
    try:
        
        x, y = keypoint['x'], keypoint['y']

        if x == 0 or y == 0:# or confidence == 0:
            return (0, 0)
        
        # subtraction = subtraction_values.get(person.lower(), 0)
        # print(f"Person: {person}, Subtraction: {subtraction}")
        # x -= subtraction
        
        x_norm = x / frame_width
        y_norm = y / frame_height
        
        return (x_norm, y_norm)
    except (ValueError, SyntaxError, TypeError):
        return (0, 0)

In [191]:
subtraction_values = {
    "person1": 60,
    "person2": 370,
    "person3": 680,
    "person4": 1030
}

def transform(keypoints_dict):

    normalized_keypoints_dict = {person: {} for person in keypoints_dict.keys()}
    
    for person, keypoints in keypoints_dict.items():
        for kp_idx, keypoint in keypoints.items():
            if keypoint['x'] == 0 and keypoint['y'] == 0:
                normalized_keypoints_dict[person][kp_idx] = (0, 0)  # Default for missing keypoints
            else:
                normalized_keypoints_dict[person][kp_idx] = parse_and_normalize(keypoint, person,subtraction_values)
    
    return normalized_keypoints_dict

In [160]:
# 1 input inference 
from tensorflow.keras.models import load_model
def main( video_path, keypoint_model_weights,classifier):

    cap = cv2.VideoCapture(video_path)

    # model = load_model(classifier)
    # model_rf = joblib.load("./rf_1in.joblib")
    model = joblib.load(classifier)
    X_test = []
    X_test_x = []
    X_test_y = []
    person_number = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        keypoints = keypoint_model(frame, keypoint_model_weights)
        # for person_keypoints in keypoints:
        #         for kp_idx, keypoint in enumerate(person_keypoints[:len(keypoint_labels)]):
        #             x, y = keypoint[0].item(), keypoint[1].item()

        normalized_keypoints = transform(keypoints)
        print("normalized", normalized_keypoints)

        # X_test = [kp_list for person in normalized_keypoints.values() for kp_list in person.values()]# for kp in kp_list]
        X_test = [kp for person in normalized_keypoints.values() for kp_list in person.values() for kp in kp_list]
        if not X_test:
            print("No keypoints detected, unable to make predictions.")
            return

        X_test = np.array(X_test).reshape(-1, 10)
        
        if len(X_test) == 0:
            print("No keypoints detected, skipping predictions.")
        else:
            predictions = model.predict(X_test)
            print("Predictions:", predictions)
        
            for i, prediction in enumerate(predictions):
                if prediction > 0.5:
                    cv2.putText(frame, "hand_raise", (180+i*280 , 200),cv2.FONT_HERSHEY_SIMPLEX, 1,(255,0,0),1)

        cv2.imshow('Pose Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [193]:
# def main2( folder_path, keypoint_model_weights,classifier):
def main2( folder_path):

    video_path = folder_path
    cap = cv2.VideoCapture(video_path)

    # model = load_model(classifier)
    model = joblib.load("C:/OsamaEjaz/Qiyas_Gaze_Estimation/Wajahat_Yolo_keypoint/rf_best1.joblib")
    X_test_x = []
    X_test_y = []
    person_number = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame,(1280,720))
        keypoints = keypoint_model(frame)#, keypoint_model_weights)
        # print("keypoints:", keypoints)
        for person, kp_idct in keypoints.items():
            for kp_id, coords in kp_idct.items():
                x,y = int(coords['x']), int(coords['y'])
                cv2.circle(frame,(x,y), 5, (0,255,0), -1)

        # points
        normalized_keypoints = transform(keypoints)
        print("normalized", normalized_keypoints)

        X_test_x = [kp_list[0] for person in normalized_keypoints.values() for kp_list in person.values()]# for kp in kp_list]
        X_test_y = [kp_list[1] for person in normalized_keypoints.values() for kp_list in person.values()]# for kp in kp_list]

        print("x test:", X_test_x, X_test_y)
 
        # Convert to NumPy arrays for model input
        # X_test_x = np.array(X_test_x).reshape(-1, 10)  # Reshape based on x-coordinates
        # X_test_y = np.array(X_test_y).reshape(-1, 10)  # Reshape based on y-coordinates

        # x_comb = np.hstack(X_test_x,X_test_y)
        X_test_combined = np.hstack((np.array(X_test_x).reshape(-1, 10), 
                                     np.array(X_test_y).reshape(-1, 10)))
        print("combined:", X_test_combined)

        predictions = model.predict(X_test_combined)
        print("Predictions:", predictions)

        for i, prediction in enumerate(predictions):

            if prediction == 1:  # Assuming a threshold for binary classification
                cv2.putText(frame, "Hand Raised", (180 + i * 280, 200),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 1)
                    # cv2.putText(frame, person_label, (50 + i * 30, 200),
                    #             cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 1)

        cv2.imshow('Pose Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [195]:
# main2("E:/Wajahat/Qiyas_new_videos_for_annottaion/Qiyas_batch_6/batch_6_16.mp4")#,"runs/pose/trail4/weights/best_y11.pt","rf_2in2.joblib")
# main2("C:/Users/LAMBDA THETA/Downloads/test_bench_02/test_bench_02/Cam_19_14.mp4")#,"runs/pose/trail4/weights/best_y11.pt","dc_gen.joblib")
main2("C:/Users/LAMBDA THETA/Downloads/test_bench_02/test_bench_02/Cam_104_batch_5_46.mp4")#, "rf_gen.joblib")
# "C:\Users\LAMBDA THETA\Downloads\handraise\11.mp4" test_bench_02/test_bench_02/Cam_104_batch_5_57


0: 384x640 3 persons, 30.2ms
Speed: 2.0ms preprocess, 30.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
keypoints {'Person1': {0: {'x': 138.57, 'y': 273.11}, 1: {'x': 154.41, 'y': 326.67}, 2: {'x': 119.24, 'y': 291.15}, 3: {'x': 181.96, 'y': 299.7}, 4: {'x': 100.99, 'y': 341.11}, 5: {'x': 118.37, 'y': 307.87}, 6: {'x': 0, 'y': 0}, 7: {'x': 237.94, 'y': 345.62}, 8: {'x': 294.09, 'y': 346.62}, 9: {'x': 0, 'y': 0}}, 'Person2': {0: {'x': 457.95, 'y': 399.98}, 1: {'x': 455.92, 'y': 434.57}, 2: {'x': 419.9, 'y': 401.73}, 3: {'x': 500.43, 'y': 404.85}, 4: {'x': 402.54, 'y': 439.47}, 5: {'x': 378.44, 'y': 429.71}, 6: {'x': 476.33, 'y': 365.12}, 7: {'x': 545.21, 'y': 433.04}, 8: {'x': 602.53, 'y': 427.22}, 9: {'x': 520.46, 'y': 369.14}}, 'Person3': {0: {'x': 850.64, 'y': 312.19}, 1: {'x': 841.38, 'y': 366.03}, 2: {'x': 802.7, 'y': 325.37}, 3: {'x': 883.84, 'y': 326.34}, 4: {'x': 764.77, 'y': 398.95}, 5: {'x': 0, 'y': 0}, 6: {'x': 0, 'y': 0}, 7: {'x': 898.59, 'y': 388.65},

c:\Anaconda\envs\ypose\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(



0: 384x640 3 persons, 15.7ms
Speed: 1.0ms preprocess, 15.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
keypoints {'Person1': {0: {'x': 138.57, 'y': 273.12}, 1: {'x': 154.4, 'y': 326.67}, 2: {'x': 119.23, 'y': 291.14}, 3: {'x': 181.96, 'y': 299.72}, 4: {'x': 100.97, 'y': 341.13}, 5: {'x': 118.34, 'y': 307.88}, 6: {'x': 0, 'y': 0}, 7: {'x': 237.94, 'y': 345.64}, 8: {'x': 294.09, 'y': 346.65}, 9: {'x': 0, 'y': 0}}, 'Person2': {0: {'x': 457.93, 'y': 399.95}, 1: {'x': 455.91, 'y': 434.58}, 2: {'x': 419.88, 'y': 401.72}, 3: {'x': 500.42, 'y': 404.87}, 4: {'x': 402.52, 'y': 439.45}, 5: {'x': 378.45, 'y': 429.68}, 6: {'x': 476.33, 'y': 365.14}, 7: {'x': 545.19, 'y': 433.05}, 8: {'x': 602.52, 'y': 427.23}, 9: {'x': 520.47, 'y': 369.15}}, 'Person3': {0: {'x': 850.64, 'y': 312.26}, 1: {'x': 841.36, 'y': 366.0}, 2: {'x': 802.71, 'y': 325.38}, 3: {'x': 883.83, 'y': 326.33}, 4: {'x': 764.77, 'y': 398.99}, 5: {'x': 0, 'y': 0}, 6: {'x': 0, 'y': 0}, 7: {'x': 898.54, 'y': 388.61}

c:\Anaconda\envs\ypose\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(



0: 384x640 3 persons, 14.9ms
Speed: 0.0ms preprocess, 14.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
keypoints {'Person1': {0: {'x': 139.6, 'y': 271.56}, 1: {'x': 155.23, 'y': 327.01}, 2: {'x': 119.64, 'y': 291.45}, 3: {'x': 182.86, 'y': 298.88}, 4: {'x': 100.94, 'y': 342.13}, 5: {'x': 117.74, 'y': 308.46}, 6: {'x': 0, 'y': 0}, 7: {'x': 238.28, 'y': 346.13}, 8: {'x': 294.19, 'y': 346.88}, 9: {'x': 0, 'y': 0}}, 'Person2': {0: {'x': 458.11, 'y': 400.18}, 1: {'x': 456.28, 'y': 435.06}, 2: {'x': 419.83, 'y': 401.88}, 3: {'x': 500.8, 'y': 405.16}, 4: {'x': 402.4, 'y': 439.88}, 5: {'x': 378.65, 'y': 429.9}, 6: {'x': 476.66, 'y': 365.28}, 7: {'x': 545.27, 'y': 433.55}, 8: {'x': 602.27, 'y': 427.5}, 9: {'x': 520.65, 'y': 368.96}}, 'Person3': {0: {'x': 850.8, 'y': 312.37}, 1: {'x': 841.4, 'y': 366.26}, 2: {'x': 802.75, 'y': 325.54}, 3: {'x': 883.96, 'y': 326.57}, 4: {'x': 764.66, 'y': 398.84}, 5: {'x': 0, 'y': 0}, 6: {'x': 0, 'y': 0}, 7: {'x': 898.55, 'y': 388.63}, 8: 

...

c:\Anaconda\envs\ypose\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(



0: 384x640 3 persons, 20.5ms
Speed: 2.0ms preprocess, 20.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
keypoints {'Person1': {0: {'x': 138.29, 'y': 263.92}, 1: {'x': 152.93, 'y': 326.74}, 2: {'x': 118.11, 'y': 287.68}, 3: {'x': 181.76, 'y': 288.76}, 4: {'x': 99.55, 'y': 341.44}, 5: {'x': 116.35, 'y': 314.37}, 6: {'x': 0, 'y': 0}, 7: {'x': 237.46, 'y': 347.0}, 8: {'x': 289.49, 'y': 312.77}, 9: {'x': 217.12, 'y': 278.79}}, 'Person2': {0: {'x': 449.46, 'y': 397.01}, 1: {'x': 446.91, 'y': 433.16}, 2: {'x': 415.08, 'y': 401.7}, 3: {'x': 496.39, 'y': 408.9}, 4: {'x': 394.51, 'y': 434.85}, 5: {'x': 376.62, 'y': 425.46}, 6: {'x': 477.33, 'y': 369.26}, 7: {'x': 540.87, 'y': 435.4}, 8: {'x': 595.56, 'y': 429.64}, 9: {'x': 523.68, 'y': 371.68}}, 'Person3': {0: {'x': 872.81, 'y': 363.34}, 1: {'x': 870.62, 'y': 407.39}, 2: {'x': 819.74, 'y': 378.84}, 3: {'x': 904.06, 'y': 366.82}, 4: {'x': 771.82, 'y': 440.82}, 5: {'x': 0, 'y': 0}, 6: {'x': 0, 'y': 0}, 7: {'x': 924.96, 'y': 

c:\Anaconda\envs\ypose\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(



0: 384x640 3 persons, 19.9ms
Speed: 1.0ms preprocess, 19.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
keypoints {'Person1': {0: {'x': 138.51, 'y': 264.32}, 1: {'x': 153.13, 'y': 326.93}, 2: {'x': 118.35, 'y': 287.83}, 3: {'x': 181.94, 'y': 289.09}, 4: {'x': 99.59, 'y': 341.63}, 5: {'x': 116.09, 'y': 314.62}, 6: {'x': 0, 'y': 0}, 7: {'x': 237.43, 'y': 347.06}, 8: {'x': 289.38, 'y': 312.57}, 9: {'x': 217.07, 'y': 278.78}}, 'Person2': {0: {'x': 450.61, 'y': 396.07}, 1: {'x': 449.54, 'y': 433.86}, 2: {'x': 416.08, 'y': 400.72}, 3: {'x': 498.01, 'y': 408.42}, 4: {'x': 395.95, 'y': 436.75}, 5: {'x': 378.39, 'y': 426.0}, 6: {'x': 475.48, 'y': 369.46}, 7: {'x': 541.8, 'y': 434.91}, 8: {'x': 595.4, 'y': 430.33}, 9: {'x': 521.75, 'y': 372.04}}, 'Person3': {0: {'x': 871.25, 'y': 363.22}, 1: {'x': 871.46, 'y': 406.95}, 2: {'x': 818.78, 'y': 377.23}, 3: {'x': 903.99, 'y': 367.09}, 4: {'x': 770.51, 'y': 440.66}, 5: {'x': 0, 'y': 0}, 6: {'x': 0, 'y': 0}, 7: {'x': 925.52, 'y':

c:\Anaconda\envs\ypose\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [ ]:
# main2("E:/Wajahat/Qiyas_new_videos_for_annottaion/Qiyas_batch_6/batch_6_05.mp4","runs/pose/trail4/weights/best_y11.pt","rf_2in2.joblib")
main2("C:/Users/LAMBDA THETA/Downloads/test_bench_02/test_bench_02/Cam_19_03.mp4")#,"runs/pose/trail4/weights/best_y11.pt","dc_gen.joblib")
# main2("C:/Users/LAMBDA THETA/Downloads/test_bench_02/test_bench_02/Cam_104_batch_5_57.mp4", "rf_gen.joblib")
# "C:\Users\LAMBDA THETA\Downloads\handraise\11.mp4" test_bench_02/test_bench_02/Cam_104_batch_5_57